# Введение в MapReduce модель на Python


In [164]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

In [165]:
def MAP(_, row:NamedTuple):  #на выходе идет пара возраст(учитываются только женщины) - и информация о пользователях с этим возрастом
  if (row.gender == 'female'):
    yield (row.age, row) # как return но возвращает генератор

def REDUCE(age:str, rows:Iterator[NamedTuple]):  #считает среднее количество социальных контактов для 1 возраста женщин
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

Модель элемента данных

In [166]:
class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

In [167]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

Функция RECORDREADER моделирует чтение элементов с диска или по сети.

In [168]:
def RECORDREADER():
  return [(u.id, u) for u in input_collection]

In [169]:
list(RECORDREADER())

[(0, User(id=0, age=55, social_contacts=20, gender='male')),
 (1, User(id=1, age=25, social_contacts=240, gender='female')),
 (2, User(id=2, age=25, social_contacts=500, gender='female')),
 (3, User(id=3, age=33, social_contacts=800, gender='female'))]

In [170]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

In [171]:
map_output = flatten(map(lambda x: MAP(*x), RECORDREADER()))
map_output = list(map_output) # materialize
map_output

[(25, User(id=1, age=25, social_contacts=240, gender='female')),
 (25, User(id=2, age=25, social_contacts=500, gender='female')),
 (33, User(id=3, age=33, social_contacts=800, gender='female'))]

In [172]:
def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

In [173]:
shuffle_output = groupbykey(map_output)
shuffle_output = list(shuffle_output)
shuffle_output

[(25,
  [User(id=1, age=25, social_contacts=240, gender='female'),
   User(id=2, age=25, social_contacts=500, gender='female')]),
 (33, [User(id=3, age=33, social_contacts=800, gender='female')])]

In [174]:
reduce_output = flatten(map(lambda x: REDUCE(*x), shuffle_output))
reduce_output = list(reduce_output)
reduce_output

[(25, 370.0), (33, 800.0)]

Все действия одним конвейером!

In [175]:
list(flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER()))))))

[(25, 370.0), (33, 800.0)]

# **MapReduce**
Выделим общую для всех пользователей часть системы в отдельную функцию высшего порядка. Это наиболее простая модель MapReduce, без учёта распределённого хранения данных.

Пользователь для решения своей задачи реализует RECORDREADER, MAP, REDUCE.

In [176]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

## Спецификация MapReduce



```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

mapreduce ((k1,v1)*) -> (k3,v3)*
groupby ((k2,v2)*) -> (k2,v2*)*
flatten (e2**) -> e2*

mapreduce .map(f).flatten.groupby(k2).map(g).flatten
```




# Примеры

## SQL

In [177]:
from typing import NamedTuple # requires python 3.6+
from typing import Iterator

class User(NamedTuple):
  id: int
  age: str
  social_contacts: int
  gender: str

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800)
]

def MAP(_, row:NamedTuple):
  if (row.gender == 'female'):
    yield (row.age, row)

def REDUCE(age:str, rows:Iterator[NamedTuple]):
  sum = 0
  count = 0
  for row in rows:
    sum += row.social_contacts
    count += 1
  if (count > 0):
    yield (age, sum/count)
  else:
    yield (age, 0)

def RECORDREADER():
  return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(25, 370.0), (33, 800.0)]

## Matrix-Vector multiplication

In [178]:
from typing import Iterator
import numpy as np

mat = np.ones((5,4))
vec = np.random.rand(4) # in-memory vector in all map tasks

def MAP(coordinates:(int, int), value:int):
  i, j = coordinates
  yield (i, value*vec[j])

def REDUCE(i:int, products:Iterator[NamedTuple]):
  sum = 0
  for p in products:
    sum += p
  yield (i, sum)

def RECORDREADER():
  for i in range(mat.shape[0]):
    for j in range(mat.shape[1]):
      yield ((i, j), mat[i,j])

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(0, 2.9455344084307593),
 (1, 2.9455344084307593),
 (2, 2.9455344084307593),
 (3, 2.9455344084307593),
 (4, 2.9455344084307593)]

## Inverted index

In [179]:
from typing import Iterator

d1 = "it is what it is"
d2 = "what is it"
d3 = "it is a banana"
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    yield ("{}".format(docid), document)

def MAP(docId:str, body:str):
  for word in set(body.split(' ')):
    yield (word, docId)

def REDUCE(word:str, docIds:Iterator[str]):
  yield (word, sorted(docIds))

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('what', ['0', '1']),
 ('it', ['0', '1', '2']),
 ('is', ['0', '1', '2']),
 ('banana', ['2']),
 ('a', ['2'])]

## WordCount

In [180]:
from typing import Iterator

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3]

def RECORDREADER():
  for (docid, document) in enumerate(documents):
    for (lineid, line) in enumerate(document.split('\n')):
      yield ("{}:{}".format(docid,lineid), line)

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[('', 3), ('it', 9), ('is', 9), ('what', 5), ('a', 1), ('banana', 1)]

# MapReduce Distributed

Добавляется в модель фабрика RECORDREARER-ов --- INPUTFORMAT, функция распределения промежуточных результатов по партициям PARTITIONER, и функция COMBINER для частичной аггрегации промежуточных результатов до распределения по новым партициям.

In [181]:
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
  global reducers
  return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

## Спецификация MapReduce Distributed


```
f (k1, v1) -> (k2,v2)*
g (k2, v2*) -> (k3,v3)*

e1 (k1, v1)
e2 (k2, v2)
partition1 (k2, v2)*
partition2 (k2, v2*)*

flatmap (e1->e2*, e1*) -> partition1*
groupby (partition1*) -> partition2*

mapreduce ((k1,v1)*) -> (k3,v3)*
mapreduce .flatmap(f).groupby(k2).flatmap(g)
```



## WordCount

In [182]:
from typing import Iterator
import numpy as np

d1 = """
it is what it is
it is what it is
it is what it is"""
d2 = """
what is it
what is it"""
d3 = """
it is a banana"""
documents = [d1, d2, d3, d1, d2, d3]

maps = 3
reducers = 2

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for (docid, document) in enumerate(split):
      for (lineid, line) in enumerate(document.split('\n')):
        yield ("{}:{}".format(docid,lineid), line)

  split_size =  int(np.ceil(len(documents)/maps))
  for i in range(0, len(documents), split_size):
    yield RECORDREADER(documents[i:i+split_size])

def MAP(docId:str, line:str):
  for word in line.split(" "):
    yield (word, 1)

def REDUCE(word:str, counts:Iterator[int]):
  sum = 0
  for c in counts:
    sum += c
  yield (word, sum)

# try to set COMBINER=REDUCER and look at the number of values sent over the network
partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

56 key-value pairs were sent over a network.


[(0, [('', 6), ('a', 2), ('banana', 2), ('it', 18), ('what', 10)]),
 (1, [('is', 18)])]

## TeraSort

In [183]:
import numpy as np

input_values = np.random.rand(30)
maps = 3
reducers = 2
min_value = 0.0
max_value = 1.0

def INPUTFORMAT():
  global maps

  def RECORDREADER(split):
    for value in split:
        yield (value, None)

  split_size =  int(np.ceil(len(input_values)/maps))
  for i in range(0, len(input_values), split_size):
    yield RECORDREADER(input_values[i:i+split_size])

def MAP(value:int, _):
  yield (value, None)

def PARTITIONER(key):
  global reducers
  global max_value
  global min_value
  bucket_size = (max_value-min_value)/reducers
  bucket_id = 0
  while((key>(bucket_id+1)*bucket_size) and ((bucket_id+1)*bucket_size<max_value)):
    bucket_id += 1
  return bucket_id

def REDUCE(value:int, _):
  yield (None,value)

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None, PARTITIONER=PARTITIONER)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
partitioned_output

30 key-value pairs were sent over a network.


[(0,
  [(None, 0.04863228526164354),
   (None, 0.05946646216390883),
   (None, 0.15091602944917382),
   (None, 0.15860291784078373),
   (None, 0.19552609907727503),
   (None, 0.2610312311523375),
   (None, 0.30270421328182584),
   (None, 0.31513920013857843),
   (None, 0.38163864130881375),
   (None, 0.3833523915796392),
   (None, 0.4203783033309483),
   (None, 0.4286830565385442),
   (None, 0.4590872989348779),
   (None, 0.46195977042128855),
   (None, 0.46236319246243796),
   (None, 0.4912399100401764),
   (None, 0.49893217999184447)]),
 (1,
  [(None, 0.5082861079387357),
   (None, 0.5163020266015491),
   (None, 0.5367451647128103),
   (None, 0.604153700745626),
   (None, 0.610768108811461),
   (None, 0.7738141115689252),
   (None, 0.7863379806757853),
   (None, 0.8367813353398721),
   (None, 0.8372461054440316),
   (None, 0.894754169063298),
   (None, 0.9356432630319088),
   (None, 0.9396438183447936),
   (None, 0.9802082906808057)])]

# Упражнения
Упражнения взяты из Rajaraman A., Ullman J. D. Mining of massive datasets. – Cambridge University Press, 2011.


Для выполнения заданий переопределите функции RECORDREADER, MAP, REDUCE. Для модели распределённой системы может потребоваться переопределение функций PARTITION и COMBINER.

### Максимальное значение ряда

Разработайте MapReduce алгоритм, который находит максимальное число входного списка чисел.

In [184]:

def MAP_MAX(_, row: NamedTuple):
    yield ("", row)

def REDUCE_MAX(_, rows: Iterator[NamedTuple]):
    max_age = 0
    user_max_age = None
    for row in rows:
        if row.age > max_age:
            max_age = row.age
            user_max_age = row
    yield user_max_age


def RECORDREADER():
    return [(u.id, u) for u in input_collection]


output = MapReduce(RECORDREADER, MAP_MAX, REDUCE_MAX)
output = list(output)
output

[User(id=0, age=55, social_contacts=20, gender='male')]

### Арифметическое среднее

Разработайте MapReduce алгоритм, который находит арифметическое среднее.

$$\overline{X} = \frac{1}{n}\sum_{i=0}^{n} x_i$$


In [185]:
def MAP_AVG(_, row: NamedTuple):
    yield ("", row)


def REDUCE_AVG(_, rows: Iterator[NamedTuple]):
    sum_avg = 0
    cnt = 0
    for row in rows:
        sum_avg += row.social_contacts
        cnt += 1
    yield (sum_avg/cnt)

def RECORDREADER():
    return [(u.id, u) for u in input_collection]


output = MapReduce(RECORDREADER, MAP, REDUCE)
output = list(output)
output

[(None, 0), (None, 1), (None, 2), (None, 3)]

### GroupByKey на основе сортировки

Реализуйте groupByKey на основе сортировки, проверьте его работу на примерах

In [186]:
def groupbykey(iterable):
    iterable = sorted(iterable, key=lambda e: e[0])
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

output = MapReduce(RECORDREADER, MAP_MAX, REDUCE_MAX)
output = list(output)
print(f"MapReduce MAX: {output}")

output = MapReduce(RECORDREADER, MAP_AVG, REDUCE_AVG)
output = list(output)
print(f"MapReduce AVG: {output}")

MapReduce MAX: [User(id=0, age=55, social_contacts=20, gender='male')]
MapReduce AVG: [390.0]


### Drop duplicates (set construction, unique elements, distinct)

Реализуйте распределённую операцию исключения дубликатов

In [187]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=0, age=25, gender='female', social_contacts=240),
    User(id=1, age=25, gender='male', social_contacts=500),
    User(id=1, age=33, gender='female', social_contacts=800),
    User(id=2, age=45, gender='male', social_contacts=617),
    User(id=2, age=17, gender='female', social_contacts=100),
    User(id=3, age=22, gender='female', social_contacts=237)
]

def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()

def groupbykey_distributed(map_partitions, PARTITIONER):
  global reducers
  partitions = [dict() for _ in range(reducers)]
  for map_partition in map_partitions:
    for (k2, v2) in map_partition:
      p = partitions[PARTITIONER(k2)]
      p[k2] = p.get(k2, []) + [v2]
  return [(partition_id, sorted(partition.items(), key=lambda x: x[0])) for (partition_id, partition) in enumerate(partitions)]

def PARTITIONER(obj):
    global reducers
    return hash(obj) % reducers

def MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, PARTITIONER=PARTITIONER, COMBINER=None):
  map_partitions = map(lambda record_reader: flatten(map(lambda k1v1: MAP(*k1v1), record_reader)), INPUTFORMAT())
  if COMBINER != None:
    map_partitions = map(lambda map_partition: flatten(map(lambda k2v2: COMBINER(*k2v2), groupbykey(map_partition))), map_partitions)
  reduce_partitions = groupbykey_distributed(map_partitions, PARTITIONER) # shuffle
  reduce_outputs = map(lambda reduce_partition: (reduce_partition[0], flatten(map(lambda reduce_input_group: REDUCE(*reduce_input_group), reduce_partition[1]))), reduce_partitions)

  print("{} key-value pairs were sent over a network.".format(sum([len(vs) for (k,vs) in flatten([partition for (partition_id, partition) in reduce_partitions])])))
  return reduce_outputs

maps = 3
reducers = 2

def INPUTFORMAT():
    global maps

    def RECORDREADER(split):
        for (userindex, user) in enumerate(split):
            yield (userindex, user)

    split_size = int(np.ceil(len(input_collection) / maps))
    for i in range(0, len(input_collection), split_size):
        yield RECORDREADER(input_collection[i : i + split_size])

def MAP_UNIQUE(_, user: str):
    yield (user.id, user)

def REDUCE_UNIQUE(user_id: int, users: Iterator[int]):
    yield users[0]

partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP_UNIQUE, REDUCE_UNIQUE, COMBINER=None)
partitioned_output = [(partition_id, list(partition)) for (partition_id, partition) in partitioned_output]
print(partitioned_output)

7 key-value pairs were sent over a network.
[(0, [User(id=0, age=55, social_contacts=20, gender='male'), User(id=2, age=45, social_contacts=617, gender='male')]), (1, [User(id=1, age=25, social_contacts=500, gender='male'), User(id=3, age=22, social_contacts=237, gender='female')])]


#Операторы реляционной алгебры
### Selection (Выборка)

**The Map Function**: Для  каждого кортежа $t \in R$ вычисляется истинность предиката $C$. В случае истины создаётся пара ключ-значение $(t, t)$. В паре ключ и значение одинаковы, равны $t$.

**The Reduce Function:** Роль функции Reduce выполняет функция идентичности, которая возвращает то же значение, что получила на вход.



In [188]:
input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='female', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800),
]

def MAP_SELECT(_, row: NamedTuple):
    if row.gender == "male":
        yield (row, row)
def REDUCE_SELECT(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)

def RECORDREADER():
    return [(u.id, u) for u in input_collection]

output = MapReduce(RECORDREADER, MAP_SELECT, REDUCE_SELECT)
output = list(output)
output

[(User(id=0, age=55, social_contacts=20, gender='male'),
  [User(id=0, age=55, social_contacts=20, gender='male')])]

### Projection (Проекция)

Проекция на множество атрибутов $S$.

**The Map Function:** Для каждого кортежа $t \in R$ создайте кортеж $t′$, исключая  из $t$ те значения, атрибуты которых не принадлежат  $S$. Верните пару $(t′, t′)$.

**The Reduce Function:** Для каждого ключа $t′$, созданного любой Map задачей, вы получаете одну или несколько пар $(t′, t′)$. Reduce функция преобразует $(t′, [t′, t′, . . . , t′])$ в $(t′, t′)$, так, что для ключа $t′$ возвращается одна пара  $(t′, t′)$.

In [189]:
input_collection = [
    dict(id=0, age=55, gender='male', social_contacts=20),
    dict(id=0, age=25, gender='female', social_contacts=240),
    dict(id=1, age=25, gender='female', social_contacts=500),
    dict(id=2, age=33, gender='female', social_contacts=800),
   dict(id=2, age=5, gender='cat', social_contacts=2),
]

def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element


def groupbykey(iterable):
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))


def MAP_PROJECTION(_, row: dict):
    S = ["male", "female"]
    if row["gender"] in S:
        yield (row["id"], row)
    else:
        modified_row = row.copy()
        del modified_row["gender"]
        yield (modified_row["id"], modified_row)

def REDUCE_PROJECTION(row: str, rows: Iterator[NamedTuple]):
    yield (row, rows)


def RECORDREADER():
    return [(u["id"], u) for u in input_collection]


output = MapReduce(RECORDREADER, MAP_PROJECTION, REDUCE_PROJECTION)
output = list(output)
output

[(0,
  [{'id': 0, 'age': 55, 'gender': 'male', 'social_contacts': 20},
   {'id': 0, 'age': 25, 'gender': 'female', 'social_contacts': 240}]),
 (1, [{'id': 1, 'age': 25, 'gender': 'female', 'social_contacts': 500}]),
 (2,
  [{'id': 2, 'age': 33, 'gender': 'female', 'social_contacts': 800},
   {'id': 2, 'age': 5, 'social_contacts': 2}])]

### Union (Объединение)

**The Map Function:** Превратите каждый входной кортеж $t$ в пару ключ-значение $(t, t)$.

**The Reduce Function:** С каждым ключом $t$ будет ассоциировано одно или два значений. В обоих случаях создайте $(t, t)$ в качестве выходного значения.

In [190]:
input_collection_a = [
    User(id=0, age=55, gender='male', social_contacts=20),
    User(id=1, age=25, gender='female', social_contacts=240),
    User(id=2, age=25, gender='male', social_contacts=500),
    User(id=3, age=33, gender='female', social_contacts=800),
]

input_collection_b = [
    User(id=3, age=33, gender='female', social_contacts=800),
    User(id=4, age=66, gender='female', social_contacts=156),
]

def MAP_UNION(_, row: NamedTuple):
    yield (row.id, row)


def REDUCE_UNION(row: str, rows: Iterator[NamedTuple]):
    yield (rows[0], rows[0])

def RECORDREADER():
    return [(u.id, u) for u in input_collection_a + input_collection_b]


output = MapReduce(RECORDREADER, MAP_UNION, REDUCE_UNION)
output = list(output)
output

[(User(id=0, age=55, social_contacts=20, gender='male'),
  User(id=0, age=55, social_contacts=20, gender='male')),
 (User(id=1, age=25, social_contacts=240, gender='female'),
  User(id=1, age=25, social_contacts=240, gender='female')),
 (User(id=2, age=25, social_contacts=500, gender='male'),
  User(id=2, age=25, social_contacts=500, gender='male')),
 (User(id=3, age=33, social_contacts=800, gender='female'),
  User(id=3, age=33, social_contacts=800, gender='female')),
 (User(id=4, age=66, social_contacts=156, gender='female'),
  User(id=4, age=66, social_contacts=156, gender='female'))]

### Intersection (Пересечение)

**The Map Function:** Превратите каждый кортеж $t$ в пары ключ-значение $(t, t)$.

**The Reduce Function:** Если для ключа $t$ есть список из двух элементов $[t, t]$ $-$ создайте пару $(t, t)$. Иначе, ничего не создавайте.

In [191]:

def MAP_INTERSECTION(_, row: NamedTuple):
    yield (row.id, row)

def REDUCE_INTERSECTION(row_id: int, rows: Iterator[NamedTuple]):
    if len(rows) == 2:
        yield rows

def RECORDREADER():
    return [(u.id, u) for u in input_collection_a + input_collection_b]

output = MapReduce(RECORDREADER, MAP_INTERSECTION, REDUCE_INTERSECTION)
output = list(output)
output

[[User(id=3, age=33, social_contacts=800, gender='female'),
  User(id=3, age=33, social_contacts=800, gender='female')]]

### Difference (Разница)

**The Map Function:** Для кортежа $t \in R$, создайте пару $(t, R)$, и для кортежа $t \in S$, создайте пару $(t, S)$. Задумка заключается в том, чтобы значение пары было именем отношения $R$ or $S$, которому принадлежит кортеж (а лучше, единичный бит, по которому можно два отношения различить $R$ or $S$), а не весь набор атрибутов отношения.

**The Reduce Function:** Для каждого ключа $t$, если соответствующее значение является списком $[R]$, создайте пару $(t, t)$. В иных случаях не предпринимайте действий.

In [192]:

def MAP_DIFFERENCE(collection_id, user):
    yield (user, collection_id)

def REDUCE_DIFFERENCE(user, collections):
    if collections == [0]:
        yield (user)

def RECORDREADER():
    return [(0, a) for a in input_collection_a] + [(1, b) for b in input_collection_b]

output = MapReduce(RECORDREADER, MAP_DIFFERENCE, REDUCE_DIFFERENCE)
output = list(output)
output

[User(id=0, age=55, social_contacts=20, gender='male'),
 User(id=1, age=25, social_contacts=240, gender='female'),
 User(id=2, age=25, social_contacts=500, gender='male')]

### Natural Join

**The Map Function:** Для каждого кортежа $(a, b)$ отношения $R$, создайте пару $(b,(R, a))$. Для каждого кортежа $(b, c)$ отношения $S$, создайте пару $(b,(S, c))$.

**The Reduce Function:** Каждый ключ $b$ будет асоциирован со списком пар, которые принимают форму либо $(R, a)$, либо $(S, c)$. Создайте все пары, одни, состоящие из  первого компонента $R$, а другие, из первого компонента $S$, то есть $(R, a)$ и $(S, c)$. На выходе вы получаете последовательность пар ключ-значение из списков ключей и значений. Ключ не нужен. Каждое значение, это тройка $(a, b, c)$ такая, что $(R, a)$ и $(S, c)$ это принадлежат входному списку значений.

In [193]:
class User(NamedTuple):
    id: int
    age: str
    gender: str
    social_contacts: int
    pet_id: int

class Pet(NamedTuple):
    id: int
    name: str

def MAP_JOIN(pet_id, row):
    yield (pet_id, row)


def REDUCE_JOIN(pet_id, rows):
    users = []
    pet = None

    for row in rows:
        if type(row) is User:
            users += [row]
        else:
            pet = row

    for row in rows:
        if type(row) is User:
            yield (row, row.pet_id, pet)

def RECORDREADER():
    return [(user.pet_id, user) for user in input_collection] + [(pet.id, pet) for pet in pet_collection]

input_collection = [
    User(id=0, age=55, gender='male', social_contacts=20, pet_id=0),
    User(id=1, age=25, gender='female', social_contacts=240,pet_id=1),
    User(id=2, age=25, gender='female', social_contacts=500,pet_id=2),
    User(id=3, age=33, gender='female', social_contacts=800,pet_id=3),
    User(id=3, age=11, gender='male', social_contacts=800,pet_id=1),
]

pet_collection = [
    Pet(id=0, name="Cat"),
    Pet(id=1, name="Dog"),
    Pet(id=2, name="Hamster"),
    Pet(id=3, name="Parrot"),
]

output = MapReduce(RECORDREADER, MAP_JOIN, REDUCE_JOIN)
output = list(output)
joined_data = output
joined_data

[(User(id=0, age=55, gender='male', social_contacts=20, pet_id=0),
  0,
  Pet(id=0, name='Cat')),
 (User(id=1, age=25, gender='female', social_contacts=240, pet_id=1),
  1,
  Pet(id=1, name='Dog')),
 (User(id=3, age=11, gender='male', social_contacts=800, pet_id=1),
  1,
  Pet(id=1, name='Dog')),
 (User(id=2, age=25, gender='female', social_contacts=500, pet_id=2),
  2,
  Pet(id=2, name='Hamster')),
 (User(id=3, age=33, gender='female', social_contacts=800, pet_id=3),
  3,
  Pet(id=3, name='Parrot'))]

### Grouping and Aggregation (Группировка и аггрегация)

**The Map Function:** Для каждого кортежа $(a, b, c$) создайте пару $(a, b)$.

**The Reduce Function:** Ключ представляет ту или иную группу. Примение аггрегирующую операцию $\theta$ к списку значений $[b1, b2, . . . , bn]$ ассоциированных с ключом $a$. Возвращайте в выходной поток $(a, x)$, где $x$ результат применения  $\theta$ к списку. Например, если $\theta$ это $SUM$, тогда $x = b1 + b2 + · · · + bn$, а если $\theta$ is $MAX$, тогда $x$ это максимальное из значений $b1, b2, . . . , bn$.

In [194]:
def MAP_GROUP(pet_id, user, pet):
    yield (pet_id, user)

def REDUCE_GROUP(pet_id, rows):
    yield f"pet with id={pet_id} has {len(rows)} user(s)"

def RECORDREADER():
    return [(pet_id, user, pet) for user, pet_id, pet in joined_data]

output = MapReduce(RECORDREADER, MAP_GROUP, REDUCE_GROUP)
output = list(output)
output

['pet with id=0 has 1 user(s)',
 'pet with id=1 has 2 user(s)',
 'pet with id=2 has 1 user(s)',
 'pet with id=3 has 1 user(s)']

#

### Matrix-Vector multiplication

Случай, когда вектор не помещается в памяти Map задачи


In [195]:
from typing import Iterator, Tuple

input_matrix = [
    (1, 1, 1), (1, 1, 1), (1, 1, 1),
    (1, 1, 1), (1, 1, 1), (1, 1, 1),
    (1, 1, 1), (1, 1, 1), (1, 1, 1),
]

input_vector = [
    (1, 1), (1, 1), (1, 1)
]


def flatten(nested_iterable):
    for iterable in nested_iterable:
        for element in iterable:
            yield element

def groupbykey(iterable):
    t = {}
    for k2, v2 in iterable:
        t[k2] = t.get(k2, []) + [v2]
    return t.items()


def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))


def MAP_MATRIX_VECTOR(_, matrix_row):
    row, col, value = matrix_row
    for vec_col, vec_value in input_vector:
        if vec_col == col:
            yield (row, value * vec_value)


def REDUCE_MATRIX_VECTOR(row, values: Iterator[int]):
    yield (row, sum(values))


def RECORDREADER():
    return [(None, m) for m in input_matrix]


output = MapReduce(RECORDREADER, MAP_MATRIX_VECTOR, REDUCE_MATRIX_VECTOR)
output = list(output)
output

[(1, 27)]

## Matrix multiplication (Перемножение матриц)

Если у нас есть матрица $M$ с элементами $m_{ij}$ в строке $i$ и столбце $j$, и матрица $N$ с элементами $n_{jk}$ в строке $j$ и столбце $k$, тогда их произведение $P = MN$ есть матрица $P$ с элементами $p_{ik}$ в строке $i$ и столбце $k$, где

$$p_{ik} =\sum_{j} m_{ij}n_{jk}$$

Необходимым требованием является одинаковое количество столбцов в $M$ и строк в $N$, чтобы операция суммирования по  $j$ была осмысленной. Мы можем размышлять о матрице, как об отношении с тремя атрибутами: номер строки, номер столбца, само значение. Таким образом матрица $M$ предстваляется как отношение $ M(I, J, V )$, с кортежами $(i, j, m_{ij})$, и, аналогично, матрица $N$ представляется как отношение $N(J, K, W)$, с кортежами $(j, k, n_{jk})$. Так как большие матрицы как правило разреженные (большинство значений равно 0), и так как мы можем нулевыми значениями пренебречь (не хранить), такое реляционное представление достаточно эффективно для больших матриц. Однако, возможно, что координаты $i$, $j$, и $k$ неявно закодированы в смещение позиции элемента относительно начала файла, вместо явного хранения. Тогда, функция Map (или Reader) должна быть разработана таким образом, чтобы реконструировать компоненты $I$, $J$, и $K$ кортежей из смещения.

Произведение $MN$ это фактически join, за которым следуют группировка по ключу и аггрегация. Таким образом join отношений $M(I, J, V )$ и $N(J, K, W)$, имеющих общим только атрибут $J$, создаст кортежи $(i, j, k, v, w)$ из каждого кортежа $(i, j, v) \in M$ и кортежа $(j, k, w) \in N$. Такой 5 компонентный кортеж представляет пару элементов матрицы $(m_{ij} , n_{jk})$. Что нам хотелось бы получить на самом деле, это произведение этих элементов, то есть, 4 компонентный кортеж$(i, j, k, v \times w)$, так как он представляет произведение $m_{ij}n_{jk}$. Мы представляем отношение как результат одной MapReduce операции, в которой мы можем произвести группировку и аггрегацию, с $I$ и $K$  атрибутами, по которым идёт группировка, и суммой  $V \times W$.





In [196]:
# MapReduce model
def flatten(nested_iterable):
  for iterable in nested_iterable:
    for element in iterable:
      yield element

def groupbykey(iterable):
  t = {}
  for (k2, v2) in iterable:
    t[k2] = t.get(k2, []) + [v2]
  return t.items()

def MapReduce(RECORDREADER, MAP, REDUCE):
  return flatten(map(lambda x: REDUCE(*x), groupbykey(flatten(map(lambda x: MAP(*x), RECORDREADER())))))

Реализуйте перемножение матриц с использованием модельного кода MapReduce для одной машины в случае, когда одна матрица хранится в памяти, а другая генерируется RECORDREADER-ом.

In [197]:
import numpy as np
I = 2
J = 3
K = 4*10
small_mat = np.random.rand(I,J) # it is legal to access this from RECORDREADER, MAP, REDUCE
big_mat = np.random.rand(J,K)

def RECORDREADER():
  for j in range(big_mat.shape[0]):
    for k in range(big_mat.shape[1]):
      yield ((j,k), big_mat[j,k])

def MAP(k1, v1):
  (j, k) = k1
  w = v1
  # solution code that yield(k2,v2) pairs
  for i in range(I):
    k2 = (i, k)
    v2 = small_mat[i, j] * w
    yield (k2, v2)


def REDUCE(key, values):
  (i, k) = key
  # solution code that yield(k3,v3) pairs
  k3 = (i, k)
  v3 = 0
  for j in range(J):
    v3 += values[j]
  yield (k3, v3)

Проверьте своё решение

In [198]:
# CHECK THE SOLUTION
reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)

def asmatrix(reduce_output):
  reduce_output = list(reduce_output)
  I = max(i for ((i,k), vw) in reduce_output)+1
  K = max(k for ((i,k), vw) in reduce_output)+1
  mat = np.empty(shape=(I,K))
  for ((i,k), vw) in reduce_output:
    mat[i,k] = vw
  return mat

np.allclose(reference_solution, asmatrix(solution)) # should return true

True

In [199]:
reduce_output = list(MapReduce(RECORDREADER, MAP, REDUCE))
max(i for ((i,k), vw) in reduce_output)

1

Реализуйте перемножение матриц  с использованием модельного кода MapReduce для одной машины в случае, когда обе матрицы генерируются в RECORDREADER. Например, сначала одна, а потом другая.

In [200]:
I = 2
J = 3
K = 4 * 10
small_mat = np.random.rand(I, J)
big_mat = np.random.rand(J, K)

def RECORDREADER():
    for i in range(I):
        for j in range(J):
            for k in range(K):
                yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))


def MAP(element1, element2):
    (i, j), v1 = element1
    (j, k), v2 = element2
    yield ((i, k), v1 * v2)

def REDUCE(key, values):
    (i, k) = key
    k3 = (i, k)
    v3 = 0
    for j in range(J):
        v3 += values[j]

    yield (k3, v3)

reference_solution = np.matmul(small_mat, big_mat)
solution = MapReduce(RECORDREADER, MAP, REDUCE)


def asmatrix(reduce_output):
    reduce_output = list(reduce_output)
    I = max(i for ((i, k), vw) in reduce_output) + 1
    K = max(k for ((i, k), vw) in reduce_output) + 1
    mat = np.empty(shape=(I, K))
    for (i, k), vw in reduce_output:
        mat[i, k] = vw
    return mat


np.allclose(reference_solution, asmatrix(solution))

True

Реализуйте перемножение матриц с использованием модельного кода MapReduce Distributed, когда каждая матрица генерируется в своём RECORDREADER.

In [201]:
maps = 2
reducers = 2


def INPUTFORMAT():
    global maps
    def RECORDREADER(i_range):
        for i in i_range:
            for j in range(J):
                for k in range(K):
                    yield (((i, j), small_mat[i, j]), ((j, k), big_mat[j, k]))

    split_size = int(np.ceil(I / maps))
    for i in range(0, I, split_size):
        yield RECORDREADER(range(i, i + split_size))


def MAP(element1, element2):
    (i, j), v1 = element1
    (j, k), v2 = element2

    yield ((i, k), v1 * v2)

def REDUCE(key, values):
    (i, k) = key
    # solution code that yield(k3,v3) pairs
    k3 = (i, k)
    v3 = 0
    for j in range(J):
        v3 += values[j]

    yield (k3, v3)


partitioned_output = MapReduceDistributed(INPUTFORMAT, MAP, REDUCE, COMBINER=None)
partitioned_output = [ (partition_id, list(partition)) for (partition_id, partition) in partitioned_output ]

solution = []
for output_part in partitioned_output:
    for element in output_part[1]:
        solution += [element]

print(asmatrix(solution))
np.allclose(reference_solution, asmatrix(solution))

240 key-value pairs were sent over a network.
[[0.60247588 0.51707122 0.35714608 0.46294631 0.70400954 0.4064662
  0.50706964 0.50611397 0.46654041 0.36272204 0.48397201 0.28930234
  0.81024561 0.96449072 0.24492202 0.28543589 0.45771435 1.04416976
  0.33131764 0.29127183 0.80630196 1.07424881 0.3538811  0.74908992
  0.43790253 0.2356519  0.20229478 0.39734556 0.6393019  0.33358229
  0.24510778 0.38904227 0.64057454 1.0354342  0.11760194 0.47286397
  0.67995254 0.71943911 0.5590594  0.32037762]
 [0.77935339 0.82563046 0.40528147 0.33891066 0.77240404 0.42280238
  0.84012882 0.72808815 0.79553733 0.36533097 0.64011631 0.5315758
  0.81887956 0.89345533 0.49606183 0.27136192 0.63069461 1.10139433
  0.34168835 0.40172464 0.81111853 1.17056123 0.64350014 0.84063401
  0.70613908 0.37919592 0.37922014 0.69623503 0.50939138 0.76712913
  0.51300748 0.27565531 0.8117922  1.08421433 0.19766295 0.59127015
  0.54668233 0.57484665 0.47517109 0.47745597]]


True

Обобщите предыдущее решение на случай, когда каждая матрица генерируется несколькими RECORDREADER-ами, и проверьте его работоспособность. Будет ли работать решение, если RECORDREADER-ы будут генерировать случайное подмножество элементов матрицы?

Да, решение будем работать, потому что в RECORDREADER у нас пары элементов идентифицируются по индексам